# Challenges

## Installs and Imports

In [8]:
!pip install torch numpy matplotlib requests torchvision


     ---------------------------------------- 1.6/1.6 MB 12.4 MB/s eta 0:00:00



[notice] A new release of pip available: 22.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.functional as F

## Challenge 1

- This challenge will test your knowledge of PyTorch tensors.
<br/>
1. Create a tensor of size 10x10 with all elements equal to 0.5.
2. Create a tensor with shape (1, 3, 3) with three `3`s, three `2`s, and three `4`s


In [ ]:
x1 = torch.tensor([])

In [ ]:
x2 = torch.tensor([])

## Challenge 2

This challenge will test how well you can apply the concepts of deep learning.
<br/>
1. You are given an input with shape (1, 3) and a model with one layer, with weights in the shape (3, 1). There is no bias.
2. The model also has the sigmoid activation function which takes the output of the weights as its input (remember what it spits out?).
3. Identity what values the weights should be if the goal of this model is to map inputs with bigger values on the left to 0, and inputs with bigger values on the right to 1.
4. Example `[1, 0, 0]` -> 0, `[1, 1, 3]` -> 1

NOTE: Applying to Deep Learning to solve this problem is silly but its a good exercise :)



In [5]:
x = torch.randn(1,3)

w1, w2, w3 = None, None, None

w = torch.tensor([w1, w2, w3], dtype=torch.float32)

act = F.sigmoid

TypeError: must be real number, not NoneType

In [ ]:
print(x)
print()

print(f'output = {act(x @ w)}')

## Challenge 3

- Run the following code.
- Be able to describe (almost all) of what is going on.
- Play around with the model, the optimizer, etc.

In [9]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [10]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100.0%


Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



100.0%


Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



100.0%


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



100.0%

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



In [11]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [17]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [18]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [19]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [20]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [21]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [22]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.298089  [   64/60000]
loss: 2.292016  [ 6464/60000]
loss: 2.268456  [12864/60000]
loss: 2.269043  [19264/60000]
loss: 2.253275  [25664/60000]
loss: 2.221361  [32064/60000]
loss: 2.233205  [38464/60000]
loss: 2.191187  [44864/60000]
loss: 2.194559  [51264/60000]
loss: 2.164777  [57664/60000]
Test Error: 
 Accuracy: 38.6%, Avg loss: 2.159276 

Epoch 2
-------------------------------
loss: 2.167047  [   64/60000]
loss: 2.163869  [ 6464/60000]
loss: 2.100232  [12864/60000]
loss: 2.121493  [19264/60000]
loss: 2.069453  [25664/60000]
loss: 2.008163  [32064/60000]
loss: 2.043052  [38464/60000]
loss: 1.952209  [44864/60000]
loss: 1.971410  [51264/60000]
loss: 1.904873  [57664/60000]
Test Error: 
 Accuracy: 49.0%, Avg loss: 1.896052 

Epoch 3
-------------------------------
loss: 1.925823  [   64/60000]
loss: 1.906212  [ 6464/60000]
loss: 1.780241  [12864/60000]
loss: 1.831648  [19264/60000]
loss: 1.721286  [25664/60000]
loss: 1.668470  [32064/600